# Graph Problems

25 Practice Problems on Graphs

---

### 1.LC.210: **Course Schedule II** | `Medium`

- [Video Solution](https://www.youtube.com/watch?v=EgI5nU9etnU&list=PLot-Xpze53ldBT_7QA8NVot219jFNr_GI&index=2)

#### Solution Thoughts 🤔

The problem describes a _dependency_ between courses. Dependencies should automatically prompt us to think in terms of a DAG's. Thinking divergently the tools we know of to operate on DAGs are

- [Y] Topological Sort
  - [N] BFS - optimality
  - [Y] DFS - possibility
  - [P] Arrival/Departure Accounting
    - BackEdge, CrossEdge, ForwardEdge
    - Iterative(med/hard) or Recursive(easy)
  - [P] Vertex Coloring
- [N] Strongly Connected Components - Classification
- [N] Prims/Kruskals MST - optimality
- [N] Eulerian Efficiency - optimality
- [N] Shortest Paths - optimality
  Given this elimination process we can be convergent now and say topological sort with a detection of a Back Edge should give us the solution we're looking for. Furthermore, a recursive dfs would be the easiest/fastest solution approach.


In [ ]:
class Solution:
    def findOrder(self, n, courses):
        adj_map = self.build_graph(n, courses)
        arr, dep, topo = [0] * n, [0] * n, []
        for course in range(n):
            if not arr[course]:
                has_cycle, _ = self.dfs(0, adj_map, course, arr, dep, topo)
                if has_cycle:
                    return []
        return topo

    def dfs(self, CLOCK, adj_map, u, arr, dep, topo):
        CLOCK += 1
        arr[u] = CLOCK
        for v in adj_map.get(u):
            if not arr[v]:
                cycle, CLOCK = self.dfs(CLOCK, adj_map, v, arr, dep, topo)
                if cycle:
                    return True, CLOCK
            elif not dep[v]:
                print("Back Edge: ", u, v)
                # cycle detected via Back-Edge
                return True, CLOCK
        CLOCK += 1
        dep[u] = CLOCK
        topo.append(u)
        return False, CLOCK

    def build_graph(self, n, edges):
        adj_map = {i: set() for i in range(n)}
        for u, v in edges:
            adj_map[u].add(v)
        return adj_map


args = {"n": 3, "edges": [[1, 0], [1, 2], [0, 1]]}
Solution().findOrder(*args.values())

Back Edge:  1 0


[]

#### Our Solution

- The above solution runs a bit slowly compared to online submissions:
  - Time: 32% faster
  - Space: 26% less

#### Recommended Solution

- The recommended solution by LeetCode is provided below. It uses the same technique of **Arrival** and **Departure** but abstracts the complexity of the numbering and extra memory to manage the entire list to something more terse; colors! The technique is called **Vertex Coloring**.
  - WHITE = Unvisited
  - GRAY = Arrived
  - BLACK = Arrived & Departed
- We could easily change the _Colors_ to be more intuitive to the approach we're already aware of: _Arrival & Departure_

### Notes

1. By default all vertces are WHITE
2. If the node is unprocessed, then call dfs on it.
3. Don't recurse further if we found a cycle already
4. An edge to a GRAY vertex represents a cycle
5. Recursion ends. We mark it as black
6. A pair [a, b] in the input represents edge from b --> a


In [19]:
from collections import defaultdict


class Solution:
    WHITE = 1  # unvisited
    GRAY = 2  # arrived
    BLACK = 3  # arrived & departed
    is_possible = True

    def findOrder(self, numCourses, prerequisites):
        self.edges = self.build_graph(prerequisites)
        self.colors = {k: Solution.WHITE for k in range(numCourses)}  # Note 1
        self.topo = []
        for u in range(numCourses):
            if self.colors[u] == Solution.WHITE:  # Note 2
                self.dfs(u)
        return self.topo[::-1] if self.is_possible else []

    def dfs(self, u):
        if not self.is_possible:  # Note 3
            return
        self.colors[u] = Solution.GRAY
        for v in self.edges.get(u, []):
            if self.colors.get(v) == Solution.WHITE:
                self.dfs(v)
            elif self.colors.get(v) == Solution.GRAY:  # Note 4
                self.is_possible = False
        self.colors[u] = Solution.BLACK  # Note 5
        self.topo.append(u)

    def build_graph(self, prereqs):
        edge_map = defaultdict(list)
        for v, u in prereqs:
            edge_map[u].append(v)
        return edge_map


args = {"n": 3, "edges": [[1, 0], [1, 2], [0, 1]]}
Solution().findOrder(*args.values())

[]

#### Nice User Solution

Credit `@Pythagoras_the_3rd` on LC.

1. Create a pre-requisite adjacency map. Key's = course, and Values = pre-requisite course.
2. Create a graph adjacency map. Key's = Pre-Requisite course. Values = the courses that depend on the Key.
3. Locate a starting node - a course without dependencies.
4. Pop off the Queue, add the Course pop'd to the `topo` result.
5. If the `topo` length == number of courses, we know we're finished.
6. Iterate over the neighbors for the given course, finding all neighbors that are blocked by the current node. Remove the current node from all those neighbor's pre-requisite lists. This signifies that we're taking the pre-requisite so it no longer blocks all the neighboring courses.
7. Looking within the pre-requisite adj_map, if a neighbor has no more pre-requisites, then we know it's safe to take that course, so add it to the Queue.
8. If we iterate over every legal course (get to the end of the Queue depth), and we haven't already returned, then we know it's impossible to take the course.


In [22]:
from collections import deque, defaultdict


class Solution:
    def findOrder(self, numCourses, prerequisites):
        topo = []
        preq = {i: set() for i in range(numCourses)}  # Note 1
        graph = defaultdict(set)  # Note 2
        for i, j in prerequisites:
            preq[i].add(j)
            graph[j].add(i)
        q = deque([k for k, v in preq.items() if not v])  # Note 3
        while q:
            course = q.popleft()
            topo.append(course)  # Note 4
            if len(topo) == numCourses:  # Note 5
                return topo
            for cor in graph[course]:  # Note 6
                preq[cor].remove(course)  # Note 7
                if not preq[cor]:
                    q.append(cor)
        return []  # Note 8


args = {"n": 3, "edges": [[1, 0], [1, 2], [0, 2]]}
Solution().findOrder(*args.values())

[2, 0, 1]

---

### 2.LC.133: **Clone A Graph** | `Medium`

- Given a reference of a node in a **connected** undirected graph.
  Return a **deep copy (clone)** of the graph.
  Each node in the graph contains a value (`int`) and a list (`List[Node]`) of its neighbors. You must return the copy of the given node as a reference to the cloned graph.

```python
class Node {
    public int val;
    public List<Node> neighbors;
}
```

<img src="https://imgur.com/m1hCiUX.png" style="max-width:500px">

#### Solution Thoughts 🤔

- Divergent Thinking
  1. Traversing a graph and copying the values we find...
     - [P] BFS seems like a good natural candidate
     - [N] DFS seems like it should also work, but it's not super intuitive to me how to manage state.
     - [P] DSU seems like it may work.
       1. We could convert the `make_set` function to clone the nodes as we find them.
       2. When we call `find(old_node.val)` if we don't find it, then we call `make_set`. We then iterate over each neighbor for `old_node` and call `union` with the parent copy we just made and each neighbor...it seems not so straight forward tho 😥
  2. Using BFS the flow could be
     1. Clone the starting node, and put on the queue. We only add cloned nodes onto the queue.
     2. Each Q pop is meant to handle the neighbors of the node. If the neighbor is in a visited map, we know it's already been cloned, so take the clone and add it to a new `cloned_neighbors` list. Once we look thru all the neighbors, we assign the pop'd node's `neighbors` attr, to `cloned_neighbors`. If any neighbor is not in the `visited` map, we make a clone, add and enqueue it, and add it to the `visited` map.
     3. <img src="https://imgur.com/nPJKjeh.png" style="max-width:500px"><br />This image shows the results of the cloned graph state, after having cloned the starting node `Sn1` and, and iterating over the neighbors of the starting node, and cloning `N2` and `N4`.
     4. <img src="https://imgur.com/j4t0a2n.png" style="max-width:300px"><br />The Queue has `N2` and `N4` ready to be pop'd. But the `visited` map, has the cloned copies.
     5. <img src="https://imgur.com/14krbKJ.png" style="max-width:500px"><br />The _Work Bench_ shows the process described. The old neighbors are shown in black, and then converted into clones (green). Those clones are appended to the _Cloned Neighbors_ list. Once all the neighbors are processed, `Sn1.neighbors` is assigned the clones list.
     6. The remaining images show the lifecycle of the clone being formed. Pay attention to how the edge relationships are being modified over time.
        - <img src="https://imgur.com/S5o7zsI.png" style="max-width:200px"> <img src="https://imgur.com/n7YBaiK.png" style="max-width:200px"> <img src="https://imgur.com/3zahcQE.png" style="max-width:200px"><br />
- Time & Space Complexity:
  1. Time = `O(|V| + |E|)`
  2. Space = `O(|E|)`


In [ ]:
from collections import deque


class Node:
    def __init__(self, val=0, neighbors=None):
        self.val = val
        self.neighbors = neighbors if neighbors is not None else []


class Solution:
    def cloneGraph(self, start_node: "Node") -> "Node":
        if not start_node:
            return start_node
        visited = {}  # visited clones
        cloned_start = Node(start_node.val, start_node.neighbors)
        visited[cloned_start.val] = cloned_start
        q = deque([cloned_start])
        while q:
            node = q.pop()
            cloned_neighbors = []
            for n in node.neighbors:
                if n.val not in visited:
                    clone = Node(n.val, n.neighbors)
                    visited[clone.val] = clone
                    cloned_neighbors.append(clone)
                    q.appendleft(clone)
                else:
                    cloned_neighbors.append(visited.get(n.val))
            node.neighbors = cloned_neighbors
        return cloned_start

#### Results

- Time: 20% faster 😅
- Space: 77% Less 🏆 (b.c. no recursion)

#### LC Solution | Recursive DFS

**Notes**:

1. Dictionary to save the visited node and it's respective clone as key and value respectively. This helps to avoid cycles.
2. If the node was already visited before. Return the clone from the visited dictionary.
3. Create a clone for the given node. Note that we don't have cloned neighbors as of now, hence [].
4. The key is original node and value being the clone node.
5. Iterate through the neighbors to generate their clones and prepare a list of cloned neighbors to be added to the cloned node.


In [ ]:
class Node(object):
    def __init__(self, val, neighbors):
        self.val = val
        self.neighbors = neighbors


class Solution(object):
    def __init__(self):
        self.visited = {}  # Note 1

    def cloneGraph(self, node):
        if not node:
            return node
        if node in self.visited:  # Note 2
            return self.visited[node]
        clone_node = Node(node.val, [])  # Note 3
        self.visited[node] = clone_node  # Note 4
        for n in node.neighbors:  # Note 5
            clone_node.neighbors.append(self.cloneGraph(n))
        return clone_node

---

### 3.LC.127: **Word Ladder** | `Hard`

- Given two words, beginWord and endWord, and a dictionary wordList, return the number of words in the shortest transformation sequence from beginWord to endWord, or 0 if no such sequence exists.
- ```python
  # Example 1
  Input: beginWord = "hit", endWord = "cog", wordList = ["hot","dot","dog","lot","log","cog"]
  Output: 5
  Explanation: One shortest transformation sequence is "hit" -> "hot" -> "dot" -> "dog" -> cog", which is 5 words long.

  # Example 2
  Input: beginWord = "hit", endWord = "cog", wordList = ["hot","dot","dog","lot","log"]
  Output: 0
  Explanation: The endWord "cog" is not in wordList, therefore there is no valid transformation sequence.
  ```

#### Divergent Thinking

1. The question want's an _Optimal_ answer:

- > **Shortest** transformation sequence

2. Optimality = BFS, Dynamic Programming, Shortest Path algo's.

- Technically dynamic programming is something like a DAG, and finding the shortest path in a DAG.

3. Brute Force - Recursive n-ary tree.
1. Given the `begin word` we can include/exclude a word from `wordList` if that `ith` letter matches the `ith letter` of the current parameter word. `hit` -> `hot` -> `h == h` so we call `f([hit, hot], [words from word list])` and `f([hit], [...rest of words from wordlist excluding 'hot'])`
1. The solution here would be exponential. Something like `Order(N^k)` where `k` is the number of choices at `k` depth in the tree which varies from level to level depenthing on the `ith` letter being analyzed.
1. Memoizing the solution


#### Solution Approach | BFS + Dynamic Neighbors

- **Time Complexity**
  1. `O(N * (L + 26 * L + M)) ~ O(N*(L+M))`
  2. For each word `(N)`, i need to look at each character with a length `L` (`N*L`) to determine if there's an edge to the end-word. Then I need to generate and compare all possible matches/neighbors of N (`M`). To generate those neighbors, i need to loop thru 26 characters, for the length of `N` (L times ~ `26 * L`).
- **Notes**
  1. Annoying edge case, to ensure work is worth being done otherwise quit early.
  2. We save the distance travelled so far for a given Q'd node. If the problem was asking us to return the actual path travelled, this second argument would be a list of node values indicating the "bread-crumb" trail we took.
  3. This logic could be done in a lot of places. However, i felt that it was most deterministically accurate to check directly after popping from the Q. This is akin to a recursive base-case check. As soon as we prepare to do some work, we first check if we've found the solution, otherwise, we continue.
  4. This is **the major takeaway** from this problem; Typically we generate a graph by calculating it's edges **before** we start a traversal. However in this problem, we're calculating the next set of edges **on-the-fly**. I believe this technique is why this problem is considered `HARD` rather than `MEDIUM`.
  5. As for HOW we're going about calculating the neighbors, this is quite interesting. Because we know we're looking for exactly a difference of 1 between word_1 and word_2, we know that the difference must exist within the actual alphabet. Meaning there's only 26 possible differences between two valid words having an edge between them. Therefore, we simply brute-force our way thru this list and generate every possible edge theoretically possible and validate the possibility against the given list of words. This is Asymptotically cheaper than looking thru every word, and comparing to every other word, every time we need to check for generate a neighbors list.
  6. If however the word list is shorter than 26, then it's cheaper to look thru every word.
  7. This is another key technqiue in this problem: we're _defining an edge by detecting an impossible edge_. If the difference in letter is greater than 1, the edge is impossible.


In [ ]:
from collections import deque


class Solution:
    def ladderLength(self, beginWord, endWord, wordList):
        word_set = set(wordList)
        if endWord not in word_set:
            return 0  # Note 1
        visited = set([beginWord])
        q = deque([(beginWord, 1)])  # Note 2
        while q:
            word, diff = q.pop()
            if self.has_edge(word, endWord):
                return diff + 1  # Note 3
            for n in self.get_neighbors(word, word_set):  # Note 4
                if n not in visited:
                    visited.add(n)
                    q.appendleft((n, diff + 1))
        return 0

    def get_neighbors(self, target, word_set):
        neighbors = []
        if len(word_set) > (26 * len(target)):  # Note 5
            for code in range(ord("a"), ord("z") + 1):
                for i in range(len(target)):
                    word = target[:i] + chr(code) + target[i + 1 :]
                    if word in word_set:
                        neighbors.append(word)
        else:
            for w in word_set:  # Note 6
                if self.has_edge(w, target):
                    neighbors.append(w)
        return neighbors

    def has_edge(self, target, word):
        diff = 0
        for i in range(len(word)):  # Note 7
            if target[i] != word[i]:
                diff += 1
                if diff > 1:
                    return False
        return diff == 1

#### LC Solution | BFS + Pre-Calculate All Neighbors

- **Time Complexity**
  1. `O((N * M) * M) ~ O(N * M^2)`
  2. `N` = The length of `wordList`
  3. `M` = The length of a neighbor word.
  4. Expanation: In the worst case, we need to look thru every Vertex `N`, and for each vertex, we need to look thru all it's neighbors (`M`). For each of these neighbors, we need to look thru each character of the neighbor (`M^2`).
- **Notes**
  1. Dictionary to hold combination of words that can be formed, from any given word, by changing one letter at a time.
  2. A Key is the generic word. A Value is a list of words which have the same intermediate generic word. Since the solution isn't asking us to return the actual list of words, then we don't need to use the 26 Alphabet character technique to specifically determine which words connect together. Rather we can use the `*` character, which improves the runtime considerably.
  3. Generate all possible neighbor words, given the current pop'd Word by looping thru every i'th value and replacing the i'th char with an `*`.
  4. Using the generated generic as a graph edge definition, we loop over all possible neighbors given the generic to find all possible neighbor nodes we could travel to.
  5. If at any point if we find what we are looking for, i.e. the end word - we can return with the answer.


In [ ]:
from collections import defaultdict, deque


class Solution(object):
    def ladderLength(self, beginWord, endWord, wordList):
        if endWord not in wordList:
            return 0
        L = len(beginWord)  # All words are same length
        all_neighbors = defaultdict(list)  # Note 1
        for word in wordList:
            for i in range(L):
                all_neighbors[word[:i] + "*" + word[i + 1 :]].append(word)  # Note 2
        visited = set([beginWord])
        queue = deque([(beginWord, 1)])
        while queue:
            current_word, diff = queue.pop()
            for i in range(L):
                edge = current_word[:i] + "*" + current_word[i + 1 :]  # Note 3
                for n in all_neighbors[edge]:  # Note 4
                    if n == endWord:
                        return diff + 1  # Note 5
                    if n not in visited:
                        visited.add(n)
                        queue.appendleft((n, diff + 1))
                all_neighbors[edge] = []
        return 0

---

### 4.LC.417: **Pacific Atlantic Water Flow** | `Medium`

- <img src="https://imgur.com/rFhj81x.png" style="max-width:500px">
- ```python
       Input: heights = [
              [1,2,2,3,5],
              [3,2,3,4,4],
              [2,4,5,3,1],
              [6,7,1,4,5],
              [5,1,1,2,4]
       ]
       Output: [
              [0,4], # row, col
              [1,3],
              [1,4],
              [2,2],
              [3,0],
              [3,1],
              [4,0]
       ]
       Explanation: The following cells can flow to the Pacific and Atlantic oceans, as shown below:
       [0,4]: [0,4] -> Pacific Ocean
              [0,4] -> Atlantic Ocean
       [1,3]: [1,3] -> [0,3] -> Pacific Ocean
              [1,3] -> [1,4] -> Atlantic Ocean
       [1,4]: [1,4] -> [1,3] -> [0,3] -> Pacific Ocean
              [1,4] -> Atlantic Ocean
       [2,2]: [2,2] -> [1,2] -> [0,2] -> Pacific Ocean
              [2,2] -> [2,3] -> [2,4] -> Atlantic Ocean
       [3,0]: [3,0] -> Pacific Ocean
              [3,0] -> [4,0] -> Atlantic Ocean
       [3,1]: [3,1] -> [3,0] -> Pacific Ocean
              [3,1] -> [4,1] -> Atlantic Ocean
       [4,0]: [4,0] -> Pacific Ocean
              [4,0] -> Atlantic Ocean
       Note that there are other possible paths for these cells to flow to the Pacific and Atlantic oceans.
  ```

#### Solution Approach

1. Starting from the Pacific Ocean border, we can enque all nodes at that border. Then we BFS as long as the adjacent node has a value >= the current nodes value.
2. Starting from the Atlantic Ocean border, we repeat step 1.
3. The result will contain 2 different visited sets. We take the logical intersection of the two sets, and we have our answer.
4. TimeComplexity: 2 _ rows _ cols ~ `O(rows * cols)`


In [25]:
class Solution:
    def pacificAtlantic(self, heights):
        rows, cols = len(heights), len(heights[0])
        p_visited = set()
        a_visited = set()
        for i in range(rows):
            self.get_nodes((i, 0), p_visited, heights)
            self.get_nodes((i, cols - 1), a_visited, heights)
        for i in range(cols):
            self.get_nodes((0, i), p_visited, heights)
            self.get_nodes((rows - 1, i), a_visited, heights)
        return p_visited & a_visited

    def get_nodes(self, start_node, visited, heights):
        visited.add(start_node)
        q = deque([start_node])
        while q:
            row, col = q.pop()
            value = heights[row][col]
            for i in [-1, 0, 1]:
                for j in [-1, 0, 1]:
                    if abs(i) == abs(j):
                        continue
                    r, c = row + i, col + j
                    if (
                        0 > r
                        or r >= len(heights)
                        or 0 > c
                        or c >= len(heights[0])
                        or heights[r][c] < value
                    ):
                        continue
                    if (r, c) not in visited:
                        node = (r, c)
                        q.appendleft(node)
                        visited.add((r, c))
        return visited


Solution().pacificAtlantic(
    [
        [1, 2, 2, 3, 5],
        [3, 2, 3, 4, 4],
        [2, 4, 5, 3, 1],
        [6, 7, 1, 4, 5],
        [5, 1, 1, 2, 4],
    ]
)

{(0, 4), (1, 3), (1, 4), (2, 2), (3, 0), (3, 1), (4, 0)}

### LeetCode Solution

- LC's solution uses DFS recursive, so there's a bit less boilerplate code, although the solution is a bit more expensive in terms of Time due to recursive call-stack.
- **Notes**
  1. Check if input is empty
  2. Initialize variables, including sets used to keep track of visited cells
  3. This cell is visited, so mark it
  4. Check all 4 directions
  5. Check if the new cell is within bounds. Check that the new cell has a higher or equal height, So that water can flow from the new cell to the old cell
  6. Check that the new cell hasn't already been visited
  7. Loop through each cell adjacent to the oceans and start a DFS
  8. Find all cells that can reach both oceans, and convert to list


In [ ]:
class Solution:
    def pacificAtlantic(self, matrix):
        if not matrix or not matrix[0]:
            return []  # Note 1
        num_rows, num_cols = len(matrix), len(matrix[0])  # Note 2
        pacific_visited = set()
        atlantic_visited = set()

        def dfs(row, col, visited):
            visited.add((row, col))  # Note 3
            for x, y in [(1, 0), (0, 1), (-1, 0), (0, -1)]:  # Note 4
                new_row, new_col = row + x, col + y
                if (
                    new_row < 0
                    or new_row >= num_rows
                    or new_col < 0
                    or new_col >= num_cols
                    or matrix[new_row][new_col] < matrix[row][col]
                ):  # Note 5
                    continue
                if (new_row, new_col) not in visited:  # Note 6
                    dfs(new_row, new_col, visited)

        for i in range(num_rows):  # Note 7
            dfs(i, 0, pacific_visited)
            dfs(i, num_cols - 1, atlantic_visited)
        for i in range(num_cols):
            dfs(0, i, pacific_visited)
            dfs(num_rows - 1, i, atlantic_visited)
        return list(pacific_visited.intersection(atlantic_visited))  # Note 8

---

### 5.LC.743: **Network Delay Time** | `Medium`

You are given a network of n nodes, labeled from 1 to n. You are also given times, a list of travel times as directed edges times[i] = (ui, vi, wi), where ui is the source node, vi is the target node, and wi is the time it takes for a signal to travel from source to target.

We will send a signal from a given node k. Return the minimum time it takes for all the n nodes to receive the signal. If it is impossible for all the n nodes to receive the signal, return -1.

<img src="https://imgur.com/Z8TR3Yi.png" style="max-width:500px">


In [19]:
class Graph:
    def __init__(self, edges, V):
        self.V = V
        self.adj_map = Graph.build_graph(V, edges)
        self.visited = set()
        self.distances = {}

    def dijkstra(self, start):
        self.visited.add(start)
        self.distances[start] = 0
        q = [start]
        while q:
            u = self.get_min(q)
            self.visited.add(u)
            for v, cost in self.adj_map.get(u):
                distance_u = self.get_distance(u)
                distance_v = self.get_distance(v)
                if distance_v > distance_u + cost:
                    self.distances[v] = distance_u + cost
                    q.append(v)

    def get_result(self, start):
        unvisited = set(list(self.adj_map.keys())) - self.visited
        max_delay = max(self.distances.values())
        return -1 if max_delay in [float("inf"), 0] or unvisited else max_delay

    def get_distance(self, vertex):
        if vertex in self.distances:
            return self.distances.get(vertex)
        self.distances[vertex] = float("inf")
        return self.distances.get(vertex)

    def get_min(self, q):
        for i in range(len(q) // 2 - 1, -1, -1):
            self.heapify(q, i, len(q))
        self.swap(q, 0, -1)
        return q.pop()

    def heapify(self, q, i, size):
        min_i, l, r = i, 2 * i + 1, 2 * i + 2
        distance_min = self.get_distance(q[min_i])
        if l < size and self.get_distance(q[l]) < distance_min:
            min_i = l
        if r < size and self.get_distance(q[r]) < distance_min:
            min_i = r
        if min_i != i:
            self.swap(q, min_i, i)
            self.heapify(q, min_i, size)

    @staticmethod
    def swap(a, l, r):
        a[l], a[r] = a[r], a[l]

    @staticmethod
    def build_graph(v, edges):
        adj_map = {i: set() for i in range(1, v + 1)}
        for u, v, c in edges:
            adj_map[u].add((v, c))
        return adj_map


class Solution:
    def networkDelayTime(self, times, n, k):
        g = Graph(times, n)
        g.dijkstra(k)
        return g.get_result(k)

---

### 6.LC.79: **Word Search** | `Medium`

Given an m x n grid of characters board and a string word, return true if word exists in the grid.

The word can be constructed from letters of sequentially adjacent cells, where adjacent cells are horizontally or vertically neighboring. The same letter cell may **not** be used more than once.

- **Example 1**

  - <img src="https://imgur.com/jBK0e4A.png" styl="max-width:500px">
  - ```
    Input: board = [["A","B","C","E"],["S","F","C","S"],["A","D","E","E"]], word = "ABCCED"
    Output: true
    ```

- **Example 2**

  - <img src="https://imgur.com/cIAbhLi.png" styl="max-width:500px">
  - ```
    Input: board = [["A","B","C","E"],["S","F","C","S"],["A","D","E","E"]], word = "SEE"
    Output: true
    ```

- **Example 3**
  - <img src="https://imgur.com/C5DK0aW.png" styl="max-width:500px">
  - ```
    Input: board = [["A","B","C","E"],["S","F","C","S"],["A","D","E","E"]], word = "ABCB"
    Output: false
    ```

#### Solution Approach

- Divergent Thinking
  1. We have a path and we want to find a specific path
  2. BFS, DFS, DSU
     - [3] DFS: Iterative
       - We traverse, and only put on the stack, letters matching the next adjacent character.
       - Time Complexity: (rows \* cols) + len(word)
       - **WONT WORK** Because we need to use backtracking. Backtracking is extremely difficult and hard to produce in an iterative solution. In recursion however, it's very simple.
     - [2] BFS
       - Because we know the adjacent letters to the previous letters.
       - We could prioritize nodes in the Queue, to be the next adjacent letter.
       - Time Complexity: (rows \* cols) + len(word)
     - 2] Adjacency Map
       - Build an adj_map, {key: value} = {(row, col): set((row, col)...)}
       - Because we have the target path, we can traverse thru the word, and look for an edge between word[n-1] to word[n]
       - Time Complexity: (rows \* cols) + len(word)
     - [1] DSU
- Convergent Thinking
  1. Because all Time Complexities are equal, then we can prioritize based on min-code for fastest implementation.
  2. Afterward, perhaps a more idiomatic approach would be

#### NOTES: Iterative DFS + Backtracking | `Failed Attempt`

1. Converting given word into dictionary where char index is key, and char is value.
2. We initiate a DFS if the current char is the starting char of the target word. Similar to a Connected Components detection technique.
3. If DFS returns True, we know the answer is found.
4. We save the board coords (i, j) of the char we enqueued last. We need these values, to detect adjacent neighbors, after pop'ing
5. We simply do a spiral detection around the target coordinates, and if i == j, then it's a diagonal so skip it.
6. If the neighbor coordinate is illegal/out-of-bounds, or if the neighbor coordinate is in bounds, but the character is not int the target word, then skip it.
7. If we get this far, we know the neighbor character is in the target word, but we need to also make sure, it's the NEXT character in the sequence. To determine what character is next in the sequence, we take the previous character's index from the stack pop, and simply increment by 1 to find the next target character. If that target character, matches the neighbor character, then we append to the stack.
8. If the previous character's index value is equal to the last index in the target word, we know we've found that last character in the word, so we can eject early.
9. If we haven't ejected early by now, then we know the DFS traversal, did not find the entire word in order.

---

10. Marking a node as visited, won't work because we need to be able to backtrack. There's quite a bit of tricks to try out, but it's extremely difficult and hard to explain. If DFS backtracking is what we need, then it's deterministically easier to simply build a recursive DFS with backtracking. When we want to backtrack, we simply pop off from the visited set, the faulty node we tried to go. The "state" of neighbors is saved in the call stack, so we can continue iterating from where we were one level up in the call stack.


In [12]:
from collections import deque


class Solution:
    def exist(self, board, word) -> bool:
        word_dict = {i: letter for i, letter in enumerate(list(word))}  # Note 1
        word_set = set(list(word))
        result = False
        for i, r in enumerate(board):
            for j, c in enumerate(r):
                if c == word[0]:  # Note 2
                    result = self.dfs(board, (i, j), word_dict, word_set)
                    if result:
                        return result  # Note 3
        return result

    def dfs(self, board, coords, word_dict, word_set):
        visited = set([coords])
        stack = deque([(*coords, 0)])  # Note 4
        while stack:
            progress = False
            r, c, prev_i = stack.pop()
            visited.add((r, c))
            if len(visited) == len(word_dict):  # Note 8
                return True
            for i in [-1, 0, 1]:
                for j in [-1, 0, 1]:
                    if abs(i) == abs(j):
                        continue  # Note 5
                    row, col = r + i, c + j
                    if (
                        0 > row
                        or row >= len(board)
                        or 0 > col
                        or col >= len(board[0])
                        or board[row][col] not in word_set
                        or (row, col) in visited
                    ):
                        continue  # Note 6
                    n_char = board[row][col]
                    target_char = word_dict.get(prev_i + 1)
                    if n_char == target_char:  # Note 7
                        visited.add((row, col))
                        stack.appendleft((row, col, prev_i + 1))
                        progress = True
            if not progress:
                visited.remove((r, c))
        return False  # Note 9


Solution().exist(
    [["A", "B", "C", "E"], ["S", "F", "E", "S"], ["A", "D", "E", "E"]], "ABCESEEEFS"
)

False

#### NOTES: Recursive DFS + Backtracking | `Success Attempt`

1. Because we're using recursion, it's easiest to attach all the persistent state to the Class instance so we can have a clean recursive function signature.
2. We initiate the dfs based on if the current character we're looking at, is equal to the first char in the target word.
3. This is where we either return True up the call-stack because we're finished, or we backtrack. Backtracking is demonstrated by removing the current row and col from the visited set. This allows us to revisit the node from some other direction in the future.
4. This is the only location where we return True: whenever we know we've found the last character in the word.


In [67]:
class Solution:
    def exist(self, board, word) -> bool:
        self.board = board
        self.word_dict = {i: letter for i, letter in enumerate(list(word))}  # Note 1
        self.word_set = set(list(word))
        self.last_ix = len(word) - 1
        self.rows = len(self.board)
        self.cols = len(self.board[0])
        # Note 1
        for i, r in enumerate(board):
            for j, c in enumerate(r):
                if c == word[0]:  # Note 2
                    if self.dfs(0, i, j, set([(i, j)])):
                        return True
        return False

    def dfs(self, ix, r, c, visited):
        if ix == self.last_ix:
            return True  # Note 4
        visited.add((r, c))
        for i, j in [(0, -1), (0, 1), (1, 0), (-1, 0)]:
            row, col = r + i, c + j
            if (
                0 > row
                or row >= self.rows
                or 0 > col
                or col >= self.cols
                or self.board[row][col] not in self.word_set
                or (row, col) in visited
            ):
                continue
            n_char = self.board[row][col]
            target_char = self.word_dict.get(ix + 1)
            if n_char == target_char:
                # Note 3
                if self.dfs(ix + 1, row, col, visited):
                    return True
                visited.remove((row, col))
        return False


Solution().exist(
    [
        ["A", "A", "A", "A", "A", "A"],
        ["A", "A", "A", "A", "A", "A"],
        ["A", "A", "A", "A", "A", "A"],
        ["A", "A", "A", "A", "A", "A"],
        ["A", "A", "A", "A", "A", "A"],
        ["A", "A", "A", "A", "A", "A"],
    ],
    "AAAAAAAAAAAAAAB",
)

False

---

### 7.LC.463: **Island Perimeter** | `Easy`

#### Solution Approach | DFS + Increment when no Neighbor


---

### 14.LC.778: **Swim in Rising Water** | `Hard`

#### Solution Approach | Dijkstra

- Divergent Thinking
  - BFS - Min-Cost path = Dijkstra. But we have to wait to pop off the Queue until t >= the next priorty value.
  - DFS? - if so, we'd need to add backtracking. We pick the smallest value node, from all of the neighbors, and if that smallest neighbor is larger than t, then we assign t to that value. If the smallest neighbor is smaller than t, then we traverse without updating t. However, this is a bit more complicated so...Dijkstra seems more straight forward?
  - DSU? - we make a union of current_cell + min_adjacent_cell. If after the union, the grid[-1][-1] location shares the same parent with the grid[0][0] location, then we should have our answer.
- Convergent Thinking
  - I'm a bit more comfortable writing the Dijkstra approach. However, TC: DSU may be better in **Best-Case**, but all choices should share the **same Worst-Case** = `O(n^2*logn)`.

1. Constraints say all values are unique, so simpler to track values, rather than coordinates, but either works.
2. `heapq` will sort on the first value, in a multi-value tuple, so we need to use the cell value in the first spot.
3. We peak (not pop) from the Q. `peak[1][2]` is the "peak'd nodes value", which compare to `t`, ensuring it's the right time add travel further. If it's not the right time, then increment `t` by 1 while we "wait".
4. Check to see if the pop'd node is the target destination: grid[-1][-1], if so, we're done
5. grid[row][col] = the next node's value & the sorting key as mentioned earlier. The `heapq` will take care of "heapifying" for us.
6. Optional techqniue: Dynamically generates the list of adjacent cells: `[(-1, 0), (0, -1), (0, 1), (1, 0)]`


In [80]:
import heapq


class Solution:
    adj_cells = [
        (i, j) for i in [-1, 0, 1] for j in [-1, 0, 1] if abs(i) != abs(j)
    ]  # Note 6

    def swimInWater(self, grid):
        n, t = len(grid), 0
        visited = set([grid[0][0]])  # Note 1
        pQ = [(grid[0][0], 0, 0)]  # Note 2
        while pQ:
            peak = pQ[0]
            if grid[peak[1]][peak[2]] > t:  # Note 3
                t += 1
                continue
            _, row, col = heapq.heappop(pQ)
            for dx, dy in [(-1, 0), (0, -1), (0, 1), (1, 0)]:
                r, c = row + dx, col + dy
                in_bounds = 0 <= r < n and 0 <= c < n
                if in_bounds and grid[r][c] not in visited:
                    if r == c == n - 1:
                        return t  # Note 4
                    heapq.heappush(pQ, (grid[r][c], r, c))  # Note 5
                    visited.add(grid[r][c])
        return t


Solution().swimInWater(
    [
        [0, 1, 2, 3, 4],
        [24, 23, 22, 21, 5],
        [12, 13, 14, 15, 16],
        [11, 17, 18, 19, 20],
        [10, 9, 8, 7, 6],
    ]
)

16

In [ ]:
# User Solution using DSU
class DSU(object):
    def __init__(self, N):
        self.par = list(range(N))
        self.rnk = [0] * N

    def find(self, x):
        if self.par[x] != x:
            self.par[x] = self.find(self.par[x])
        return self.par[x]

    def union(self, x, y):
        p1, p2 = self.find(x), self.find(y)
        if p1 != p2:
            if self.rnk[p1] < self.rnk[p2]:
                p1, p2 = p2, p1
            self.par[p2] = p1
            self.rnk[p1] += 1


class Solution:
    REACHABLE = 1

    def swimInWater(self, grid):
        d, N = {}, len(grid)
        for i, j in product(range(N), range(N)):
            d[grid[i][j]] = (i, j)
        dsu = DSU(N * N)
        grid = [[0] * N for _ in range(N)]
        for i in range(N * N):
            _x, _y = d[i]
            grid[_x][_y] = self.REACHABLE
            for dx, dy in [[0, 1], [0, -1], [1, 0], [-1, 0]]:
                x, y = _x + dx, _y + dy
                out_of_bounds = N > x >= 0 and N > y >= 0
                if not out_of_bounds and grid[x][y] == 1:
                    dsu.union(x * N + y, _x * N + _y)

            if dsu.find(0) == dsu.find(N * N - 1):
                return i

---

### 8.LC.1466: **Reorder Routes to Make All Paths Lead to City** | `Medium`

##### Solution Approach | DSU

1. Sink - we want to point all nodes to the sink.
2. DSU
   - we want `0` to be the last parent
   - we build the original graph
   - we start with node zero as the parent & rank: Infinity
   - because some nodes will have no parents, we'll need to use a connected component detection technique
3. Algo Steps
   - Build graph
   - Traverse through the graph one-connected component at a time.
   - Enque the start node
   - Pop from the Q:
     - if parent of node is not 0, then perform union. increment counter
     - Union the pop'd node's set, with 0.
   - Enque the neighbors of the pop'd node.


In [6]:
from collections import deque
from giant_input import giant_args


class DSU:
    def __init__(self, n):
        self.parents = [None] * n
        self.rank = [0] * n
        self.rank[0] = float("inf")

    def make_set(self, v):
        self.parents[v] = v
        self.rank[v] = 1

    def find_parent(self, v):
        if self.parents[v] == v:
            return v
        self.parents[v] = self.find_parent(self.parents[v])
        return self.parents[v]

    def union(self, v1, v2):
        p1, p2 = self.find_parent(v1), self.find_parent(v2)
        if p1 != p2:
            if self.rank[p2] > self.rank[p1]:
                p1, p2 = p2, p1
            self.parents[p2] = p1
            self.rank[p1] += 1


class Solution:
    def minReorder(self, n, connections):
        dsu = DSU(n)
        for i in range(n):
            dsu.make_set(i)
        count = 0
        q = deque(connections)
        while q:
            u, v = q.pop()
            if 0 in [v, dsu.find_parent(v)]:  # no count: road is pointing to 0
                dsu.union(0, u)
            elif 0 in [
                u,
                dsu.find_parent(u),
            ]:  # add count: road was pointing away from 0
                count += 1
                dsu.union(u, v)
            # else:
            #     q.appendleft([u, v])
        return count


Solution().minReorder(6, [[0, 1], [1, 3], [2, 3], [4, 0], [4, 5]])
# Solution().minReorder(*giant_args.values())

1

In [ ]:
int minReorder(int n, vector<vector<int>>& connections) {
    parent=vector<int>(n);
    size=vector<int>(n);
    for(int i=0;i<n;i++){
        make_set(i);
    }
    for(int i=0;i<connections.size();i++){
        if(connections[i][1]==0){
            union_set(connections[i][1],connections[i][0]);
        }
    }
    int count=0;
        while(size[0]<=n-1){
            for(int i=0;i<connections.size();i++){
                if(find_par(connections[i][1])==0){
                    union_set(connections[i][1],connections[i][0]);
                }
                else if(find_par(connections[i][0])==0){
                    count++;
                    union_set(connections[i][0],connections[i][1]);
                }
            }
        }
        return count;
    }

---
### 9.LC.261: **Graph Valid Tree** | `Easy`

##### Solution Approach | Vertex Coloring or BackEdge detection
---

### 10.LC.684: **Redundant Connections** | `Medium`

#### Solution Approach | Visited Repeat

---

### 11.LC.212: **Word Search II** | `Hard`

#### Solution Approach | Connected Components + Caching?

---

### 12.LC.269: **Alien Dictionary** | `Hard`

#### Solution Approach | Topological Sorting + Pre-Post Visit

---

### 13.LC.1584: **Min Cost to Connect All Points** | `Medium`

#### Solution Approach | Dijkstra + Every Unvisited Neighbor

---

### 15.LC.286: **Walls & Gates** | ``

#### Solution Approach |

---

### 16.LC.130: **Surrounded Regions** | ``

#### Solution Approach |

---

### 17.LC.787: **Cheapest Flights Within K Stops** | ``

#### Solution Approach |

---

### 18.LC.695: **Max Area of Island** | ``

#### Solution Approach |

---

### 19.LC.332: **Reconstruct Itinerary** | ``

#### Solution Approach |

---

### 20.LC.994: **Rotting Oranges** | ``

#### Solution Approach |

---

### 21.LC.329: **Longest Increasing Path in a Matrix** | ``

#### Solution Approach |

---

### 22.LC.909: **Snakes & Ladders** | ``

#### Solution Approach |

---

### 23.LC.752: **Open the Lock** | ``

#### Solution Approach |

---

### 24.LC.934: **Shortest Bridge** | ``

#### Solution Approach |

---

### 25.LC.802: **Find Eve-----------ntual Safe States** | ``

#### Solution Approach |
